<a href="https://colab.research.google.com/github/FaisalWant/kafka/blob/main/kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

setup Kafka:


In [11]:
!curl -sSOL https://downloads.apache.org/kafka/3.2.0/kafka_2.13-3.2.0.tgz
!tar -xzf kafka_2.13-3.2.0.tgz

In [12]:
!cd kafka_2.13-3.2.0


In [14]:
!ls

kafka_2.13-3.2.0  kafka_2.13-3.2.0.tgz	sample_data


In [15]:
!./kafka_2.13-3.2.0/bin/zookeeper-server-start.sh   -daemon   ./kafka_2.13-3.2.0/config/zookeeper.properties
!./kafka_2.13-3.2.0/bin/zookeeper-server-start.sh   -daemon    ./kafka_2.13-3.2.0/config/server.properties
!echo "Waiting for 10 secs until kafka and zookeeper services are up and running"
!sleep 10


Waiting for 10 secs until kafka and zookeeper services are up and running


In [ ]:
!ps -ef | grep kafka